# Tasks

> FrankenUI Tasks Example

In [104]:
#| default_exp tasks

In [105]:
#| hide
from nbdev.showdoc import *

In [106]:
#| export
from fasthtml.common import *
import fasthtml.common as fh
from fh_frankenui.core import *
from fh_frankenui.components import *
from fasthtml.svg import *
import json

In [107]:
#| hide
#| eval: false
from utils import create_server
app, rt = fh.fast_app(pico=False, hdrs=Theme.blue.headers())
server, Show = create_server(app)

In [108]:
%%html
<script src="https://cdn.tailwindcss.com"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js"></script>
<script type="module" src="https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js"></script>
<link rel="stylesheet" href="https://unpkg.com/franken-wc@0.0.6/dist/css/blue.min.css">

<style>
#notebook-container { max-width: none; }
.output_html * { list-style-type: none !important; }
</style>

## Data

We store data in Python to minic the static data in the examples.  Eventually this should be replaced with sqlite data store to allow for interactive functionality

In [109]:
#| export
with open('data/status_list.json', 'r') as f: data     = json.load(f)
with open('data/statuses.json',    'r') as f: statuses = json.load(f)

In [110]:
#| export
def _create_tbl_data(d):
    return {'Done': d['selected'], 'Task': d['id'], 'Title': d['title'], 
            'Status'  : d['status'], 'Priority': d['priority'] }
    
data = [_create_tbl_data(d)  for d in data]

In [111]:
#| export
priority_dd = [{'priority': "low", 'count': 36 }, {'priority': "medium", 'count': 33 }, {'priority': "high", 'count': 31 }]

rows_per_page_dd = [10,20,30,40,50]

status_dd = [{'status': "backlog", 'count': 21 },{'status': "todo", 'count': 21 },{'status': "progress", 'count': 20 },{'status': "done",'count': 19 },{'status': "cancelled", 'count': 19 }]

## Tasks Page Functions

In [112]:
#| export
def create_hotkey_li(hotkey): return NavCloseLi(A(cls='justify-between')(hotkey[0], Span(hotkey[1], cls=TextFont.muted_sm)))

hotkeys_a = (('Profile','⇧⌘P'),('Billing','⌘B'),('Settings','⌘S'),('New Team',''))
hotkeys_b = (('Logout',''), )

In [113]:
#| export
avatar_opts = DropDownNavContainer(
    NavHeaderLi(P('sveltecult'),NavSubtitle('leader@sveltecult.com')),
    NavDividerLi(),
    *map(create_hotkey_li, hotkeys_a),
    NavDividerLi(),
    *map(create_hotkey_li, hotkeys_b),)

In [114]:
#| export
def CreateTaskModal():
    return Modal(
        Div(cls='p-6')(
            ModalTitle('Create Task'),
            P('Fill out the information below to create a new task', cls=TextFont.muted_sm),
            Br(),
            Form(cls='space-y-6')(
                Grid(Div(Select(*map(Option,('Documentation', 'Bug', 'Feature')), label='Task Type', id='task_type')),
                     Div(Select(*map(Option,('In Progress', 'Backlog', 'Todo', 'Cancelled', 'Done')), label='Status', id='task_status')),
                     Div(Select(*map(Option, ('Low', 'Medium', 'High')), label='Priority', id='task_priority'))),
                TextArea(label='Title', placeholder='Please describe the task that needs to be completed'),
                DivRAligned(
                    ModalCloseButton('Cancel', cls=ButtonT.ghost),
                    ModalCloseButton('Submit', cls=ButtonT.primary),
                    cls='space-x-5'))),
        id='TaskForm')

In [115]:
#| export
page_heading = DivFullySpaced(cls='space-y-2')(
            Div(cls='space-y-2')(
                H2('Welcome back!'),P("Here's a list of your tasks for this month!", cls=TextFont.muted_sm)),
            Div(DiceBearAvatar("sveltcult",8,8),avatar_opts))

In [116]:
show(page_heading)

In [117]:
#| export
table_controls =(Input(cls='w-[250px]',placeholder='Filter task'),
     Button("Status"),
     DropDownNavContainer(map(NavCloseLi,[A(DivFullySpaced(P(a['status']), P(a['count'])),cls=TextT.capitalize) for a in status_dd])), 
     Button("Priority"),
     DropDownNavContainer(map(NavCloseLi,[A(DivFullySpaced(LAlignedIconTxt(a['priority'], icon="check"), a['count']),cls=TextT.capitalize) for a in priority_dd])),
     Button("View"),
     DropDownNavContainer(map(NavCloseLi,[A(LAlignedIconTxt(o, icon="check")) for o in ['Title','Status','Priority']])),
     Button('Create Task',cls=(ButtonT.primary, TextFont.bold_sm), uk_toggle="target: #TaskForm"))

In [118]:
Show(table_controls)

## Route Data

In [119]:
#| export
def task_dropdown():
    return Div(Button(UkIcon('ellipsis')),
               DropDownNavContainer(
                   map(NavCloseLi,[
                           A('Edit',),
                           A('Make a copy'),
                           A('Favorite',),
                           A(SpacedPP('Delete', '⌘⌫'))])))

In [120]:
#| export
def header_render(col):
    cls = 'p-2 ' + 'uk-table-shrink' if col in ('Done','Actions') else ''
    match col:
        case "Done":    return Th(CheckboxX(), cls=cls)
        case 'Actions': return Th("",       cls=cls)
        case _:         return Th(col, cls=cls)

In [125]:
#| export
def cell_render(col, val):
    def _Td(*args,cls='', **kwargs): return Td(*args, cls=f'p-2 {cls}',**kwargs)
    match col:
        case "Done": return _Td(shrink=True)(CheckboxX(selected=val))
        case "Task":  return _Td(val)
        case "Title": return _Td(cls='max-w-[500px] truncate', expand=True)(val, cls='font-medium')
        case "Status" | "Priority": return _Td(cls='uk-text-nowrap uk-text-capitalize')(Span(val))
        case "Actions": return _Td(cls='uk-table-shrink')(task_dropdown())
        case _: raise ValueError(f"Unknown column: {col}")

In [126]:
#| export
task_columns = ["Done", 'Task', 'Title', 'Status', 'Priority', 'Actions']

tasks_table = Div(cls='uk-overflow-auto mt-4 rounded-md border border-border')(
    TableFromDicts(
        header_data=task_columns,
        body_data=data,
        body_cell_render=cell_render,
        header_cell_render=header_render,
    sortable=True))


In [82]:
Show(tasks_table)

In [54]:
#| export

def footer():
    hw_cls = 'h-4 w-4'
    return DivFullySpaced(cls='mt-4 px-2 py-2')(
        Div('1 of 100 row(s) selected.', cls='flex-1 text-sm text-muted-foreground'),
        Div(cls='flex flex-none items-center space-x-8')(
            DivCentered('Page 1 of 10', cls='w-[100px] text-sm font-medium'),
            DivLAligned(
                UkIconLink(icon='chevrons-left', button=True),
                UkIconLink(icon='chevron-left', button=True),
                UkIconLink(icon='chevron-right', button=True),
                UkIconLink(icon='chevrons-right', button=True))))

In [55]:
Show(footer())

In [56]:
#| export
tasks_ui = Div(
    DivFullySpaced(cls='mt-8')(
        Div(cls='flex flex-1 gap-4')(table_controls)),
    tasks_table,
    footer(),)

In [57]:
#| export
tasks_homepage = CreateTaskModal(), Div(cls='p-8')(page_heading, tasks_ui)

In [58]:
#| hide
import nbdev; nbdev.nbdev_export()